# Image processing project's report
##### Yousri Aboudaoud M1 ISI, N° Student: 28712992


## Eight direction Sobel edge detector for brain tumor detection

### I. Introduction 
This is a suggested prototype to implement the methodology described in the article "Comparative Analysis of Eight Direction Sobel Edge Detection
Algorithm for Brain Tumor MRI Images" to detect brain tumors using MRI images, and that will be used in Deep Learning based classification.


### II. The algorithm


In [1]:
import numpy as np
import cv2
import os

In [2]:
# Function for Gaussian filtering
def gaussian_filter(image, kernel_size=3, sigma=1):
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)

First, a convolution function is created to apply the given kernel onto the image, while adding zeros to the image (padding):

In [3]:
# Function to perform convolution
def convolution(image, kernel):
    height, width = image.shape
    kernel_size = kernel.shape[0]
    padding = kernel_size // 2

    # Add zero padding to the image
    padded_image = np.pad(image, padding, mode='constant', constant_values=0)
    
    # Initialize the output image
    output = np.zeros((height, width), dtype=np.float32)
    
    # Perform convolution
    for i in range(height):
        for j in range(width):
            output[i, j] = np.sum(padded_image[i:i+kernel_size, j:j+kernel_size] * kernel)
    
    return output

The used kernel are defined as such from the paper:

<img src="EightSobelEdgeDetectionMatrices.png">

In [4]:
# Function to perform Sobel edge detection in eight directions
def sobel_edge_detection(image):
    # Define Sobel kernels for eight directions
    # Following the paper's definition
    sobel_d1 = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    sobel_d2 = np.array([[0, 1, 2], [-1, 0, 1], [-2, -1, 0]])
    sobel_d3 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, -1]])
    sobel_d4 = np.array([[-2, -1, 0], [-1, 0, 1], [0, 1, 2]])
    sobel_d5 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    sobel_d6 = -1*sobel_d4 
    sobel_d7 = -1*sobel_d3
    sobel_d8 = -1*sobel_d2
    
    sobel_kernels = [sobel_d1,sobel_d2,sobel_d3,sobel_d4,sobel_d5,sobel_d6,sobel_d7,sobel_d8]
    
    
    # Initialize list to store gradient images for each direction
    gradient_images = []
    
    # Compute gradients for each direction
    for kernel in sobel_kernels:
        gradient_images.append(convolution(image, kernel))
    
    return gradient_images

In a conventional BiDirectional Sobel edge detector, the kernels can be applied separately to the input image, to produce separate measurements of the gradient component in each orientation (call these Gx and Gy). These can then be combined together to find the absolute magnitude of the gradient at each point and the orientation of that gradient. The gradient magnitude is given by: $|G|=\sqrt{G_x^2+G_y^2}$

However, in the suggested method, 8 dimensions are suggested, hence, the magnitude is computed as a "cumulative sum of each gradient" with respect to its corresponding dimension, and that's done using the function **compute_gradient_magnitude(gradient_images)**


In [5]:
# Function to compute the magnitude of the gradient
def compute_gradient_magnitude(gradient_images):
    # Initialize an empty array to store the magnitude of the gradient
    magnitude = np.zeros_like(gradient_images[0])
    
    # Compute the magnitude of the gradient pixel-wise
    for edge in gradient_images:
        magnitude += edge**2  # Accumulate squared gradient values
    
    # Take the square root to obtain the magnitude
    magnitude = np.sqrt(magnitude)
    
    return magnitude.astype(np.uint8)

This is the code that : 
   - Loads the dataset from a data folder
   - Creates an output folder to store in it the output of the algorithm
   - Goes through all the images of the dataset using a for loop
   - Read a given image and applies the following pipeline : gaussian filtering (Preprocessing), applies the sobel kernels to the image, combines the gradients by computing their magnitude, and finally adds the output images in the output folder

In [22]:

# Path to the directory containing brain tumor images (in folder containning a reduced number of images)
input_dir = "Data_set_Brain_Tumors_Kaggle\Testing\meningioma_Reduced"

# Output directory to save the edge-detected images
output_dir = "output_directory_test4"

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Get the current working directory
cwd = os.getcwd()

# Construct the full path to the input directory
input_full_path = os.path.join(cwd, input_dir)

# Iterate through the images in the input directory
for filename in os.listdir(input_full_path):

    # Read the image
    image_path = os.path.join(input_full_path, filename)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Preprocess the image with Gaussian filtering
    preprocessed_image = gaussian_filter(image)
    
    # Perform Sobel edge detection in eight directions
    gradient_images = sobel_edge_detection(preprocessed_image)

    # Combine the results of Sobel edge detection in eight directions
    edge_image = compute_gradient_magnitude(gradient_images)

    # Thresholding the image 
    _,thresholded_image = cv2.threshold(edge_image, 180,250, cv2.THRESH_BINARY)

    # Save the edge-detected image
    output_path = os.path.join(output_dir, "edge_" + filename)
    cv2.imwrite(output_path, thresholded_image)

print("Edge detection completed. Results saved in", output_dir)


Edge detection completed. Results saved in output_directory_test4


In [23]:
cv2.imshow("Thresholded Image", thresholded_image)
cv2.waitKey(0)
cv2.destroyAllWindows()